In [ ]:
# Name: example_analysis_data_creation.ipynb
# uthors: Stephan Meighen-Berger
# Example how to construct a data set to use for later analyses

In [ ]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
import pickle

In [ ]:
# Adding path to module
sys.path.append("../")

In [ ]:
# picture path
PICS = '../pics/'

In [ ]:
# Module imports
from fourth_day import Fourth_Day, config

In [ ]:
# Settings used in this analysis
config['scenario']['population size'] = 100
config['scenario']['duration'] = 6000
config['scenario']['organism movement'] = False
config['scenario']['exclusion'] = True
config['scenario']['injection']['rate'] = 1
config['scenario']['injection']['y range'] = [0., 10.]
config['scenario']['light prop'] = {
            "switch": True,
            "x_pos": 5.,
            "y_pos": 10.,
        }
config['scenario']['detector'] = {
    "switch": True,
    "type": "PMTSpec2",
    "response": True,
    "acceptance": "Flat",
    "mean detection prob": 0.5
}
# Organisms 
config['organisms']['emission fraction'] = 0.1
config['organisms']['alpha'] = 2.
# Geometry
config['geometry']['volume'] = {
    'function': 'rectangle',
    'x_length': 26.,
    'y_length': 10.,
    'offset': None,
}
config['geometry']['observation'] = {
    'function': 'rectangle',
    'x_length': 26.,
    'y_length': 10.,
    "offset": np.array([0., 0.]),
}
config['geometry']["exclusion"] = {
    "function": "sphere",
    "radius": 0.15,
    "x_pos": 2.,
    "y_pos": 5.,
}
# Water
config['water']['model']['name'] = 'custom' # 'potential cylinder'
config['water']['model']['directory'] = "../data/current/long_run/"
config['water']['model']['time step'] = 0.1
config['advanced']['starting step'] = 0
# Detector
config['geometry']["detector properties"]["PMTSpec2"] = {
    "x_pos": 2.,
    "y_pos": 5.,
    "det num": 12, #12 pmts numbered by position 
    "x_offsets": np.array(
        [0.1,0.,-0.1,0., 0.12,-0.12,-0.12,0.12, 0.2,-0.04,-0.2,0.04]
    ) / 2., #test radius 0.3 meter, real radius 0.15 meter
    "y_offsets": np.array(
        [0.,0.1,0.,-0.1,  0.12,0.12,-0.12,-0.12, 0.04,0.2,-0.04,-0.2]
    ) / 2.,
    "opening angle": 360., # Allowing for a full view of the surroundings
    "wavelength acceptance": [ #position number,center wavelength,quantum efficiency
        [395., 405.,0.26], #0,400
        [505., 515.,0.16], #1,510
        [420., 430.,0.28], #2,425
        [465., 475.,0.23], #3,470
        [300., 600.,1.], #4,no filter
        [487., 497.,0.1], #5,492
        [540., 560.,0.1], #6,550
        [515., 535.,0.13], #7,525
        [475., 485.,0.2], #8,480
        [445., 455.,0.2], #9,450
        [455., 465.,0.23], #10,460
        [325., 375.,0.3], #11,350                                     
    ],
}

In [ ]:
# Which detectors we want to use
wavelengths = {
    "Detector 1": ["1", "#4575b4"],
    "Detector 5": ["2", "#91bfdb"],
    "Detector 8": ["3", "#e0f3f8"],
    "Detector 3": ["4", "#fee090"],
    "Detector 10": ["5", "#fc8d59"],
}

In [ ]:
# Launching multiple simulations to use in the analysis
seeds = np.arange(100)[1:]
print(seeds)
counter = 0
for seed in tqdm(seeds):
    # General
    config["general"]["random state seed"] = seed
    # Creating a fourth_day object
    fd = Fourth_Day()
    # Launching solver
    fd.sim()
    # Fetching relevant data
    # Totals
    total = fd.measured["Detector 1"].values
    for detector in wavelengths.keys():
        if detector == "Detector 1":
            continue
        fd.measured[detector].values
        total += fd.measured[detector].values
    storage_dic = {
        "time": fd.t,
        "data": total
    }
    pickle.dump(storage_dic, open("../data/storage/vort_run_%d.p" % counter, "wb"))
    counter += 1